In [1]:
!pip uninstall -qqy jupyterlab kfp  # Remove unused conflicting packages
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"
!pip install ipywidgets==7.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 16.7 MB/s eta 

In [2]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

In [3]:
from Bio import Entrez
import nltk
import re

nltk.download('punkt')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.5 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:

# Configure Entrez email
Entrez.email = "abiodunadebisi614@gmail.com"  # Replace with your email for Entrez access

# Search and fetch PubMed abstracts related to mental health disorders
def fetch_pubmed_abstracts(query, max_results=10):
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
    record = Entrez.read(handle)
    id_list = record["IdList"]
    handle.close()

    abstracts = []
    if id_list:
        handle = Entrez.efetch(db="pubmed", id=",".join(id_list), rettype="abstract", retmode="text")
        abstracts = handle.read().split("\n\n")
        handle.close()
    return abstracts

# Fetch sample data
abstracts = fetch_pubmed_abstracts("depression OR psychosis OR anxiety")
len(abstracts), abstracts[:2]


(78,
 ['1. Syst Rev. 2025 May 1;14(1):96. doi: 10.1186/s13643-025-02850-y.',
  'Prevalence and risk factors for loneliness among individuals with diabetes: a \nsystematic review and meta-analysis.'])

In [5]:

def clean_text(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"[^a-zA-Z0-9.,;:!?()\-\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

cleaned_abstracts = [clean_text(abs) for abs in abstracts if abs.strip()]
cleaned_abstracts[:2]


['1. Syst Rev. 2025 May 1;14(1):96. doi: 10.1186s13643-025-02850-y.',
 'Prevalence and risk factors for loneliness among individuals with diabetes: a systematic review and meta-analysis.']

In [6]:
import nltk
from nltk.tokenize import sent_tokenize
# Download the necessary Punkt data for tab-separated values
nltk.download('punkt_tab')
nltk.download('punkt')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:

from nltk.tokenize import sent_tokenize

def chunk_text(text, max_length=500):
    sentences = sent_tokenize(text)
    chunks, current_chunk = [], ""
    for sent in sentences:
        if len(current_chunk) + len(sent) <= max_length:
            current_chunk += " " + sent
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sent
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

text_chunks = []
for doc in cleaned_abstracts:
    text_chunks.extend(chunk_text(doc))

len(text_chunks), text_chunks[:3]


(130,
 ['1. Syst Rev. 2025 May 1;14(1):96. doi: 10.1186s13643-025-02850-y.',
  'Prevalence and risk factors for loneliness among individuals with diabetes: a systematic review and meta-analysis.',
  'Wicaksana AL(1)(2), Apriliyasari RW(3), Tsai PS(4)(5)(6).'])

# 📘 Phase 2: Embedding and FAISS Vector Store


Now that we have cleaned and chunked the text data, we convert each chunk into vector embeddings using a pre-trained model from `sentence-transformers`. Then, we store the vectors in **FAISS**, a high-performance similarity search library.
    

In [8]:

!pip install -q faiss-cpu sentence-transformers
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load a lightweight biomedical transformer
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Encode the chunks
embeddings = model.encode(text_chunks, show_progress_bar=True)
embeddings = np.array(embeddings).astype("float32")
embeddings.shape


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.1 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

(130, 384)

In [9]:

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
index.ntotal


130

In [10]:

# Query with a new sentence
query = "drugs for treating severe depression"
query_vector = model.encode([query]).astype("float32")

# Search
top_k = 5
distances, indices = index.search(query_vector, top_k)

print("🔎 Top retrieved chunks:")
for idx in indices[0]:
    print("-", text_chunks[idx])


🔎 Top retrieved chunks:
- Depression und Herpes Zoster.
- BACKGROUND: Major Depressive Disorder (MDD) has a 50-80 recurrence rate highlighting the urgent need for more efficient recurrence prevention programs. Currently, recurrences are often identified too late, while existing preventive strategies may not sufficiently address ethio-patho-physiological mechanisms for recurrence.
- In gut microbiota composition, we found that depression and anxiety were characterized by an enrichment of pro-inflammatory bacteria and a depletion of anti-inflammatory SCFAs-producing bacteria. Specifically, Actinobacteria, Proteobacteria, Rikenellaceae, Porphyromonadaceae and Bifidobacteriaceae were more abundant in the depression group, as well as Firmicutes, Prevotellacea and Ruminococcaceae in lower abundance.
- METHODS: Here we describe the protocol for the Smartphone-based Monitoring and cognition Modification Against Recurrence of Depression (SMARD) study, aiming to investigate different forms of co

# 📘 Phase 3: Retrieval-Augmented Generation (RAG) with Gemini 2.0 Flash


In this phase, we combine semantic retrieval using FAISS with **Gemini 2.0** to generate grounded, high-quality responses to questions on drug targets for depression, psychosis, and anxiety.

We simulate a RAG pipeline that:
1. Accepts a user query.
2. Retrieves top-k relevant text chunks from the FAISS index.
3. Constructs a prompt for Gemini 2.0 including retrieved context.
4. Generates an answer grounded in the retrieved PubMed knowledge.

> **Note**: We simulate Gemini 2.0 here using the `google.generativeai` package.
    

In [11]:

import google.generativeai as genai
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import os

# Load model and index again (if needed in runtime)
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Gemini configuration
genai.configure(api_key="MY_CAPSTONE_SECRET_KEY")
gemini = genai.GenerativeModel("gemini-2.0-flash")


### 🔄 Load Existing Vector Store

In [12]:

# Load your existing index (in real-world, index should be stored/persisted)
# Recreate index here for simplicity

# Load model and index again (if needed in runtime)
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

embeddings = model.encode(text_chunks).astype("float32")

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


In [13]:

def retrieve_relevant_chunks(query, top_k=5):
    query_vector = model.encode([query]).astype("float32")
    distances, indices = index.search(query_vector, top_k)
    return [text_chunks[i] for i in indices[0]]

def generate_rag_response(query):
    context_chunks = retrieve_relevant_chunks(query)
    context = "\n".join(context_chunks)
    prompt = f"You are a biomedical research assistant helping identify drug targets.\n\nContext:\n{context}\n\nUser Question: {query}\n\nAnswer:"

    response = gemini.generate_content(prompt)
    return response.text


In [14]:

query = "What are the latest drug targets for treating depression?"
response = generate_rag_response(query)
print("🤖 Gemini 2.0 Response:\n")
print(response)


BadRequest: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.

In [ ]:
# This code is not executable as it is a PDF file content.
# To execute any code from this PDF, you need to extract the Python parts and run them separately.

# Example of how to potentially extract Python code from a string
# (This requires adjustments based on the specific PDF content)

# import re

# pdf_content = """
# ... your pdf content here ...
# """
# This code is not executable as it is a PDF file content.
# To execute any code from this PDF, you need to extract the Python parts and run them separately.

# Example of how to potentially extract Python code from a string
# (This requires adjustments based on the specific PDF content)

# import re

# pdf_content = """
# ... your pdf content here ...
# """
# This code is not executable as it is a PDF file content.
# To execute any code from this PDF, you need to extract the Python parts and run them separately.

# Example of how to potentially extract Python code from a string
# (This requires adjustments based on the specific PDF content)

# import re

# pdf_content = """
# ... your pdf content here ...
# """

# python_code = re.findall(r"
# << /Type /Pages /Kids [3 0 R] /Count 1 >>
# endobj
# 3 0 obj
# << /Type /Page /Parent 2 0 R /MediaBox [0 0 595 842] /Resources <<
#   /Font <<
#     /F1 4 0 R
#     /F2 5 0 R
#   >>
# >> /Contents 6 0 R >>
# endobj
# 4 0 obj
# << /Type /Font /Subtype /Type1 /BaseFont /Helvetica >>
# endobj
# 5 0 obj
# << /Type /Font /Subtype /Type1 /BaseFont /Courier >>
# endobj
# 6 0 obj
# << /Length 21692 >>
# stream
# BT
# /F1 24 Tf
# 100 790 Td
# (AI-Powered Insights for Mental Health Drug Targets) Tj
# /F1 16 Tf
# 0 -30 Td
# (Mental Health: Depression, Psychosis & Anxiety) Tj
# 0 -40 Td
# /Times-Roman 12 Tf
# (This document presents a comprehensive AI-powered research assistant for mental health drug target discovery.) Tj
# 0 -20 Td
# (-------------------------------------------------------------------------------) Tj
# 0 -30 Td
# /The Problem Statement:/ Tj
# 0 -20 Td
# (Mental health disorders such as depression, psychosis, and anxiety pose global health challenges, especially in low-resource settings.) Tj
# 0 -20 Td
# (Manual workflows limit the identification of potential biological drug targets due to fragmented literature and time-consuming analysis.) Tj
# 0 -40 Td
# /The Project Objective:/ Tj
# 0 -20 Td
# (Develop an AI-powered research assistant that:) Tj
# 0 -20 Td
# (- Rapidly searches thousands of mental health-related scientific papers from PubMed.) Tj
# 0 -20 Td
# (- Generates insights using Retrieval-Augmented Generation (RAG).) Tj
# 0 -20 Td
# (- Employs context-aware reasoning agents for improved drug target suggestions.) Tj
# 0 -40 Td
# /The Technology Stack Includes:/ Tj
# 0 -20 Td
# (- Python, BioPython Entrez API for data fetching.) Tj
# 0 -20 Td
# (- Transformer models (Sentence-Transformers) for embedding and vector search.) Tj
# 0 -20 Td
# (- FAISS for similarity search indexing.) Tj
# 0 -20 Td
# (- Google Gemini 2.0 for generative AI completions.) Tj
# 0 -40 Td
# /The Workflow:/ Tj
# 0 -20 Td
# (1. Data gathering: PubMed abstracts extraction and preprocessing.) Tj
# 0 -20 Td
# (2. Text chunking: breaking abstracts into manageable information units.) Tj
# 0 -20 Td
# (3. Embedding & indexing: encoding text chunks and building FAISS index.) Tj
# 0 -20 Td
# (4. RAG pipeline: generic retrieval + generative answer formulation with Gemini.) Tj
# 0 -40 Td
# /The Code Overview:/ Tj
# 0 -20 Td
# // Imports and Setup
# 1 Tf
# 0 -20 Td
# /ET
# BT
# /F2 10 Tf
# 100 505 Td
# """, pdf_content) # This is a placeholder, needs to be replaced by correct regex
# This code is not executable as it is a PDF file content.
# To execute any code from this PDF, you need to extract the Python parts and run them separately.

# Example of how to potentially extract Python code from a string
# (This requires adjustments based on the specific PDF content)

import re

pdf_content = """
... your pdf content here ...
"""

# Placeholder for the actual regex.  This needs to be replaced with correct regex
# to extract python code from the pdf content string
python_code = re.findall(r"some_regex_pattern", pdf_content)

print(python_code) #print extracted code
#     /F2 5 0 R
#   >>
# >> /Contents 6 0 R >>
# endobj
# 4 0 obj
# << /Type /Font /Subtype /Type1 /BaseFont /Helvetica >>
# endobj
# 5 0 obj
# << /Type /Font /Subtype /Type1 /BaseFont /Courier >>
# endobj
# 6 0 obj
# << /Length 21692 >>
# stream
# BT
# /F1 24 Tf
# 100 790 Td
# (AI-Powered Insights for Mental Health Drug Targets) Tj
# /F1 16 Tf
# 0 -30 Td
# (Mental Health: Depression, Psychosis & Anxiety) Tj
# 0 -40 Td
# /Times-Roman 12 Tf
# (This document presents a comprehensive AI-powered research assistant for mental health drug target discovery.) Tj
# 0 -20 Td
# (-------------------------------------------------------------------------------) Tj
# 0 -30 Td
# /The Problem Statement:/ Tj
# 0 -20 Td
# (Mental health disorders such as depression, psychosis, and anxiety pose global health challenges, especially in low-resource settings.) Tj
# 0 -20 Td
# (Manual workflows limit the identification of potential biological drug targets due to fragmented literature and time-consuming analysis.) Tj
# 0 -40 Td
# /The Project Objective:/ Tj
# 0 -20 Td
# (Develop an AI-powered research assistant that:) Tj
# 0 -20 Td
# (- Rapidly searches thousands of mental health-related scientific papers from PubMed.) Tj
# 0 -20 Td
# (- Generates insights using Retrieval-Augmented Generation (RAG).) Tj
# 0 -20 Td
# (- Employs context-aware reasoning agents for improved drug target suggestions.) Tj
# 0 -40 Td
# /The Technology Stack Includes:/ Tj
# 0 -20 Td
# (- Python, BioPython Entrez API for data fetching.) Tj
# 0 -20 Td
# (- Transformer models (Sentence-Transformers) for embedding and vector search.) Tj
# 0 -20 Td
# (- FAISS for similarity search indexing.) Tj
# 0 -20 Td
# (- Google Gemini 2.0 for generative AI completions.) Tj
# 0 -40 Td
# /The Workflow:/ Tj
# 0 -20 Td
# (1. Data gathering: PubMed abstracts extraction and preprocessing.) Tj
# 0 -20 Td
# (2. Text chunking: breaking abstracts into manageable information units.) Tj
# 0 -20 Td
# (3. Embedding & indexing: encoding text chunks and building FAISS index.) Tj
# 0 -20 Td
# (4. RAG pipeline: generic retrieval + generative answer formulation with Gemini.) Tj
# 0 -40 Td
# /The Code Overview:/ Tj
# 0 -20 Td
# // Imports and Setup
# 1 Tf
# 0 -20 Td
# /ET
# BT
# /F2 10 Tf
# 100 505 Td




%PDF-1.4
%âãÏÓ
1 0 obj
<< /Type /Catalog /Pages 2 0 R >>
endobj
2 0 obj
<< /Type /Pages /Kids [3 0 R] /Count 1 >>
endobj
3 0 obj
<< /Type /Page /Parent 2 0 R /MediaBox [0 0 595 842] /Resources <<
  /Font <<
    /F1 4 0 R
    /F2 5 0 R
  >>
>> /Contents 6 0 R >>
endobj
4 0 obj
<< /Type /Font /Subtype /Type1 /BaseFont /Helvetica >>
endobj
5 0 obj
<< /Type /Font /Subtype /Type1 /BaseFont /Courier >>
endobj
6 0 obj
<< /Length 21692 >>
stream
BT
/F1 24 Tf
100 790 Td
(AI-Powered Insights for Mental Health Drug Targets) Tj
/F1 16 Tf
0 -30 Td
(Mental Health: Depression, Psychosis & Anxiety) Tj
0 -40 Td
/Times-Roman 12 Tf
(This document presents a comprehensive AI-powered research assistant for mental health drug target discovery.) Tj
0 -20 Td
(-------------------------------------------------------------------------------) Tj
0 -30 Td
/The Problem Statement:/ Tj
0 -20 Td
(Mental health disorders such as depression, psychosis, and anxiety pose global health challenges, especially in low-resource settings.) Tj
0 -20 Td
(Manual workflows limit the identification of potential biological drug targets due to fragmented literature and time-consuming analysis.) Tj
0 -40 Td
/The Project Objective:/ Tj
0 -20 Td
(Develop an AI-powered research assistant that:) Tj
0 -20 Td
(- Rapidly searches thousands of mental health-related scientific papers from PubMed.) Tj
0 -20 Td
(- Generates insights using Retrieval-Augmented Generation (RAG).) Tj
0 -20 Td
(- Employs context-aware reasoning agents for improved drug target suggestions.) Tj
0 -40 Td
/The Technology Stack Includes:/ Tj
0 -20 Td
(- Python, BioPython Entrez API for data fetching.) Tj
0 -20 Td
(- Transformer models (Sentence-Transformers) for embedding and vector search.) Tj
0 -20 Td
(- FAISS for similarity search indexing.) Tj
0 -20 Td
(- Google Gemini 2.0 for generative AI completions.) Tj
0 -40 Td
/The Workflow:/ Tj
0 -20 Td
(1. Data gathering: PubMed abstracts extraction and preprocessing.) Tj
0 -20 Td
(2. Text chunking: breaking abstracts into manageable information units.) Tj
0 -20 Td
(3. Embedding & indexing: encoding text chunks and building FAISS index.) Tj
0 -20 Td
(4. RAG pipeline: generic retrieval + generative answer formulation with Gemini.) Tj
0 -40 Td
/The Code Overview:/ Tj
0 -20 Td
// Imports and Setup
1 Tf
0 -20 Td
/ET
BT
/F2 10 Tf
100 505 Td


In [ ]:
!pip install streamlit

In [18]:
import os
import re
import numpy as np
import pandas as pd
import nltk
import faiss
import streamlit as st
from Bio import Entrez
from sentence_transformers import SentenceTransformer
import google.generativeai as genai # This is palm2

from google import genai as google_genai #Renamed to avoid conflict
from google.genai import types

google_genai.__version__ # Using renamed module

# Assuming you have your API key stored in an environment variable named 'MY_CAPSTONE_SECRET_KEY'
os.environ["GOOGLE_API_KEY"] = "my_day1_secret_access_key"  # Replace with your actual API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"]) # Use genai.configure with your api key instead of set_api_key


In [1]:
# 1. Install dependencies
!pip install streamlit==1.22.0
!pip install pyngrok
!pip install Bio
!pip install faiss-cpu
!pip install sentence_transformers
!pip install google-generativeai
!pip install nltk -q faiss-cpu streamlit pyngrok

!pip install -U pyngrok
!ngrok update

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.22.0 requires protobuf<4,>=3.12, but you have protobuf 5.29.4 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.32.1 requires protobuf<6.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you

In [2]:

# Step 2: Import necessary libraries
import os
import re
import numpy as np
import pandas as pd
import nltk
import faiss
import streamlit as st
from Bio import Entrez
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from pyngrok import ngrok
import threading
import time


In [ ]:
# Configure Gemini AI API key
GEMINI_API_KEY = "my_day1_secret_access_key"  # Replace with your actual Gemini API key
genai.configure(api_key=GEMINI_API_KEY)

In [3]:


# Configure Entrez email
Entrez.email = "adebisiabiodun614@gmail.com"  # Replace with your email

# Step 4: Function to fetch PubMed abstracts
def fetch_pubmed_abstracts(query, max_results=10):
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
    record = Entrez.read(handle)
    id_list = record["IdList"]
    handle.close()

    abstracts = []
    if id_list:
        handle = Entrez.efetch(db="pubmed", id=",".join(id_list), rettype="abstract", retmode="text")
        abstracts = handle.read().split("\n\n")
        handle.close()
    return abstracts

# Step 5: Function to clean text
def clean_text(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"[^a-zA-Z0-9.,;:!?()\\-\\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# Step 6: Function to chunk text
def chunk_text(text, max_length=500):
    sentences = nltk.tokenize.sent_tokenize(text)
    chunks, current_chunk = [], ""
    for sent in sentences:
        if len(current_chunk) + len(sent) <= max_length:
            current_chunk += " " + sent
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sent
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# Step 7: Data Gathering & Preprocessing
def gather_data():
    abstracts = fetch_pubmed_abstracts("depression OR psychosis OR anxiety")
    cleaned_abstracts = [clean_text(abs) for abs in abstracts if abs.strip()]
    text_chunks = []
    for doc in cleaned_abstracts:
        text_chunks.extend(chunk_text(doc))
    return text_chunks

# Step 8: Embedding and FAISS Vector Store
def create_faiss_index(text_chunks):
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    embeddings = model.encode(text_chunks, show_progress_bar=False).astype("float32")
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index, model

# Step 9: Retrieve top chunks for query
def retrieve_top_chunks(query, index, model, top_k=5):
    query_vector = model.encode([query]).astype("float32")
    distances, indices = index.search(query_vector, top_k)
    return [text_chunks[i] for i in indices[0]], distances

# Step 10: Generate RAG response using Gemini AI
def generate_rag_response(query, index, model):
    retrieved_chunks, _ = retrieve_top_chunks(query, index, model)
    context = "\n".join(retrieved_chunks)
    prompt = f"""You are an expert biomedical researcher. Based on the context below, respond to the query with accurate and relevant information only. If the answer is not in the context, say so.

    Context:
    {context}

    Query:
    {query}
    """
    genai.configure(api_key=genai)
    response = genai.GenerativeModel("gemini-2.0-flash").generate_content(prompt)
    return response.text.strip()


In [4]:
# This is a full example script to run Streamlit on Google Colab with ngrok tunnel

# 2. Write a minimal Streamlit app to a file (app.py)
%%writefile app.py

app_code = """
import streamlit as st

st.title("🧠 GenAI Insights for Mental Health")
st.write("This is a demo Streamlit app running on Google Colab with ngrok tunnel.")

user_query = st.text_input("Enter your query (e.g., latest drugs for depression):")
if user_query:
    st.write(f"Searching for: {user_query}")
    # Simulate a response for demo purposes
    st.success("🔍 This is where Gemini 2.0 RAG response would appear.")
"""

with open("app.py", "w") as f:
    f.write(app_code)


Overwriting app.py


In [13]:
import threading
import os
import time
from pyngrok import ngrok

# Configure ngrok with your auth token
ngrok.set_auth_token("2wXFBKmjNctexnFA6odE6p5zBLR_44X6ZtRX1vR38YLm1JVz")  # Replace with your actual auth token

# Function to run Streamlit
def run_streamlit():
    os.system("streamlit run app.py --server.port 8501")

# Start Streamlit in a separate thread
thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()
time.sleep(5)  # Wait for Streamlit to start

# Set up ngrok tunnel programmatically
public_url = ngrok.connect(addr="8501", proto="http", name="mytunnel")

print(f"Streamlit app public URL: {public_url}")

Streamlit app public URL: NgrokTunnel: "https://cbbf-34-125-81-219.ngrok-free.app" -> "http://localhost:8501"


In [38]:
%%writefile app.py

import streamlit as st
from Bio import Entrez
import re
import nltk
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import google.generativeai as genai

# Download NLTK data
nltk.download("punkt")

# Configure Entrez email
Entrez.email = "adebisiabiodun614@gmail.com"  # Replace with your email

# Configure Gemini AI API key
GEMINI_API_KEY = "CAPSTONE_SECRET_KEY"  # Replace with your actual Gemini API key
genai.configure(api_key=GEMINI_API_KEY)

# Step 4: Function to fetch PubMed abstracts
def fetch_pubmed_abstracts(query, max_results=10):
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
    record = Entrez.read(handle)
    id_list = record["IdList"]
    handle.close()

    abstracts = []
    if id_list:
        handle = Entrez.efetch(db="pubmed", id=",".join(id_list), rettype="abstract", retmode="text")
        abstracts = handle.read().split("\n\n")
        handle.close()
    return abstracts

# Step 5: Function to clean text
def clean_text(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"[^a-zA-Z0-9.,;:!?()\\-\\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# Step 6: Function to chunk text
def chunk_text(text, max_length=500):
    sentences = nltk.tokenize.sent_tokenize(text)
    chunks, current_chunk = [], ""
    for sent in sentences:
        if len(current_chunk) + len(sent) <= max_length:
            current_chunk += " " + sent
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sent
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# Step 7: Data Gathering & Preprocessing
def gather_data():
    abstracts = fetch_pubmed_abstracts("depression OR psychosis OR anxiety")
    cleaned_abstracts = [clean_text(abs) for abs in abstracts if abs.strip()]
    text_chunks = []
    for doc in cleaned_abstracts:
        text_chunks.extend(chunk_text(doc))
    return text_chunks

# Step 8: Embedding and FAISS Vector Store
def create_faiss_index(text_chunks):
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    embeddings = model.encode(text_chunks, show_progress_bar=False).astype("float32")
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index, model

# Step 9: Retrieve top chunks for query
def retrieve_top_chunks(query, index, model, top_k=5):
    query_vector = model.encode([query]).astype("float32")
    distances, indices = index.search(query_vector, top_k)
    # Access text_chunks from st.session_state
    return [st.session_state.text_chunks[i] for i in indices[0]], distances

# Step 10: Generate RAG response using Gemini AI
def generate_rag_response(query, index, model):
    retrieved_chunks, _ = retrieve_top_chunks(query, index, model)
    context = "\n".join(retrieved_chunks)
    prompt = f"""You are an expert biomedical researcher. Based on the context below, respond to the query with accurate and relevant information only. If the answer is not in the context, say so.

    Context:
    {context}

    Query:
    {query}
    """
    response = genai.GenerativeModel("gemini-1.5-flash").generate_content(prompt)
    return response.text.strip()

# Streamlit app
st.title("🧠 GenAI Insights for Mental Health")
st.write("This is a demo Streamlit app running on Google Colab with ngrok tunnel.")

# Initialize the RAG pipeline (run once when the app starts)
if "index" not in st.session_state or "model" not in st.session_state or "text_chunks" not in st.session_state:
    with st.spinner("Initializing RAG pipeline..."):
        text_chunks = gather_data()
        if not text_chunks:
            st.error("No data retrieved from PubMed. Please check your query or internet connection.")
        else:
            index, model = create_faiss_index(text_chunks)
            st.session_state.text_chunks = text_chunks
            st.session_state.index = index
            st.session_state.model = model

# Get the user query
user_query = st.text_input("Enter your query (e.g., latest drugs for depression):")
if user_query:
    st.write(f"Searching for: {user_query}")
    with st.spinner("Generating response..."):
        try:
            response = generate_rag_response(user_query, st.session_state.index, st.session_state.model)
            st.success(response)
        except Exception as e:
            st.error(f"Error generating response: {str(e)}")

Overwriting app.py


In [39]:
import threading
import os
import time
from pyngrok import ngrok

# Configure ngrok with your auth token
ngrok.set_auth_token("2wXRBo2lluBtsSWQAjxKWTyHvNE_avHtfVS7WE4LzT2Mo5jb")  # Replace with your actual auth token

# Function to run Streamlit
def run_streamlit():
    os.system("streamlit run app.py --server.port 8501")

# Start Streamlit in a separate thread
thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()
time.sleep(5)  # Wait for Streamlit to start

# Set up ngrok tunnel programmatically
public_url = ngrok.connect(addr="8501", proto="http", name="mytunnel")

print(f"Streamlit app public URL: {public_url}")

Streamlit app public URL: NgrokTunnel: "https://7152-34-125-81-219.ngrok-free.app" -> "http://localhost:8501"


In [37]:
import google.generativeai as genai

GEMINI_API_KEY = "CAPSTONE_SECRET_KEY"  # Replace with your key
genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel("gemini-2.0-flash")
response = model.generate_content("Hello, how are you?")
print(response.text)

BadRequest: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.

In [20]:
abstracts = fetch_pubmed_abstracts("depression OR psychosis OR anxiety")
print(abstracts)

['1. J Neural Eng. 2025 May 2;22(3). doi: 10.1088/1741-2552/adcfc8.', 'A depression detection approach leveraging transfer learning with single-channel \nEEG.', 'Sun C(1), Guan M(1), Duan K(1), Gao S(1), Chen Z(1).', "Author information:\n(1)Institute of Artificial Intelligence, Anhui University of Science and \nTechnology, No. 168, Taifeng Street, Huainan 232001, Anhui, People's Republic of \nChina.", 'Objective.Major depressive disorder (MDD) is a widespread mental disorder that \naffects health. Many methods combining electroencephalography (EEG) with machine \nlearning or deep learning have been proposed to objectively distinguish between \nMDD and healthy individuals. However, most current methods detect depression \nbased on multichannel EEG signals, which constrains its application in daily \nlife. The context in which EEG is obtained can vary in terms of study designs \nand EEG equipment settings, and the available depression EEG data is limited, \nwhich could also potentially 

In [27]:
# 6. To stop ngrok and streamlit tunnel, use:
ngrok.disconnect(public_url)
ngrok.kill()